In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import networkx
import matplotlib.pyplot as plt
import numpy as np

from ws_crl_lite.datasets.dataset import build_manual_datasets

In [3]:
train, dcitrain, val, test = build_manual_datasets([100000, 100000, 10000, 10000], "atomic_markov1", 42)

/network/scratch/d/david-a.dobre/.conda/envs/weakly-supervised-causal/lib/python3.8/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
100%|██████████| 10000/10000 [00:10<00:00, 943.92it/s]


In [4]:
def save(data_file, dataset, format="normal"):
    data = dataset[:]
    if format=="normal":
        intervention_labels = \
            np.packbits(data[3], bitorder='big', axis=2) >> (8 - data[3].shape[2])
        torch.save((*data[:2], data[3], intervention_labels.squeeze()), data_file)
    elif format=="og":
        intervention_labels = data[2].squeeze()
        intervention_labels[intervention_labels==5] = 2
        intervention_labels[intervention_labels==7] = 3
        torch.save(
            (
                data[0][:, 0], 
                data[0][:, 1],
                data[1][:, 0], 
                data[1][:, 1], 
                intervention_labels.long(),
                data[3].squeeze()
            ),
            data_file
        )

    else:
        raise NotImplementedError


In [5]:
!mkdir -p ../results_scaling_markov1/data/scaling_custom/dim_3_mode_random_seed_42/

In [6]:
save("../data/3d_markov1_train.pt", train)
save("../data/3d_markov1_dci_train.pt", dcitrain)
save("../data/3d_markov1_val.pt", val)
save("../data/3d_markov1_test.pt", test)

save("../results_scaling_markov1/data/scaling_custom/dim_3_mode_random_seed_42/train.pt", train, format="og")
save("../results_scaling_markov1/data/scaling_custom/dim_3_mode_random_seed_42/dci_train.pt", dcitrain, format="og")
save("../results_scaling_markov1/data/scaling_custom/dim_3_mode_random_seed_42/val.pt", val, format="og")
save("../results_scaling_markov1/data/scaling_custom/dim_3_mode_random_seed_42/test.pt", test, format="og")



# Load their dataset

Convert it to our visualization tool

In [7]:
theirs = torch.load("../results_scaling_markov1/data/scaling/dim_3_mode_random_seed_42/test.pt")

In [13]:
torch.save(
    (
        torch.permute(torch.stack([theirs[0], theirs[1]]), (1,0,2)),
        torch.permute(torch.stack([theirs[2], theirs[3]]), (1,0,2)),
        theirs[5][:, None, :], 
        theirs[4]
    ), 
    "../data/3d_markov1_theirs.pt"
)